In [1]:
import pandas as pd
import os
import requests

os.environ['NO_PROXY'] = '127.0.0.1'

In [2]:
from fastai.vision import *
ml_path = 'C:/Users/Nico/Documents/BIH/Homeoffice/Barzooka/Covid_preprints'
learn = load_learner(path='./', file='barzooka.pkl')

C:\Users\Nico\Anaconda3\envs\fastai\lib\site-packages\torch\serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.loss.BCEWithLogitsLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\Nico\Anaconda3\envs\fastai\lib\site-packages\torch\serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\Nico\Anaconda3\envs\fastai\lib\site-packages\torch\serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source cod

### Functions for bar graph detection per PDF

In [3]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#re_pg = re.compile(r'Index \d+ out of bounds for length (\d+)')
re_pg = re.compile(r'Index: \d+, Size: (\d+)')


def req_internal(url):
    http = urllib3.PoolManager(cert_reqs='CERT_NONE')
    page = http.request('get', url, timeout=120)
    return page.data.decode('utf-8')

def count_pages(paper_id, year):
    """cantaloupe iiif server returns the highest page index with an error
    if out of range is requested
    """
    url = "http://127.0.0.1:8182/iiif/2/{}:{}.pdf/full/500,/0/default.jpg?page=1000"
    url = url.format(year, paper_id)
    page = req_internal(url)
    try:
        count = re_pg.findall(page)[0]
    except:
        count = 0
        
    return int(count)

In [51]:
def detect_graph_types_from_iiif(paper_id, year, learner, debug=False):
    """Pull images from iiif server
    """
    pages = count_pages(paper_id, year)
    if pages == 0:
        return empty_result(paper_id, year)

    url = "http://127.0.0.1:8182/iiif/2/{}:{}.pdf/full/560,560/0/default.png?page={}"
    try:
        #some PDFs can have loading problems
        images = [open_image(io.BytesIO(requests.get(url.format(year, paper_id, pg)).content)) for pg in range(1, pages+1)]
    except:
        return empty_result(paper_id, year)
    
    classes_detected = detect_graph_types_from_list(images, learner)
    classes_detected['paper_id'] = paper_id.replace("%2b", "/")
    classes_detected['year'] = year
    
    return classes_detected


def empty_result(paper_id, year):
    classes_detected = dict()  
    classes_detected['bar'] = 0
    classes_detected['pie'] = 0
    classes_detected['hist'] = 0
    classes_detected['bardot'] = 0
    classes_detected['box'] = 0
    classes_detected['dot'] = 0
    classes_detected['violin'] = 0
    classes_detected['paper_id'] = paper_id.replace("%2b", "/")
    classes_detected['year'] = year
    
    return classes_detected


def detect_graph_types_from_list(images, learner):
    """Predicts graph types for each image and returns pages with bar graphs
    """
    page_predictions = np.array([predict_graph_type(images[idx], learner) for idx in range(0, len(images))])
    bar_pages = np.where(page_predictions == 'bar')[0] + 1 #add 1 to page idx such that page counting starts at 1
    pie_pages = np.where(page_predictions == 'pie')[0] + 1
    hist_pages = np.where(page_predictions == 'hist')[0] + 1
    bardot_pages = np.where(page_predictions == 'bardot')[0] + 1
    box_pages = np.where(page_predictions == 'box')[0] + 1
    dot_pages = np.where(page_predictions == 'dot')[0] + 1
    violin_pages = np.where(page_predictions == 'violin')[0] + 1
    positive_pages = hist_pages.tolist() + bardot_pages.tolist() + box_pages.tolist() + dot_pages.tolist() + violin_pages.tolist()
    if len(positive_pages) > 0:
        positive_pages = list(set(positive_pages)) #remove duplicates and sort
        positive_pages.sort()


    classes_detected = dict()
    classes_detected['bar'] = len(bar_pages.tolist())
    classes_detected['pie'] = len(pie_pages.tolist())
    classes_detected['hist'] = len(hist_pages.tolist())
    classes_detected['bardot'] = len(bardot_pages.tolist())
    classes_detected['box'] = len(box_pages.tolist())
    classes_detected['dot'] = len(dot_pages.tolist())
    classes_detected['violin'] = len(violin_pages.tolist())

    """
    classes_detected = [len(bar_pages.tolist()),
                        len(pie_pages.tolist()),
                        len(hist_pages.tolist()),
                        len(bardot_pages.tolist()),
                        len(box_pages.tolist()),
                        len(dot_pages.tolist()),
                        len(violin_pages.tolist())                        
                       ]
    """

    return classes_detected


def predict_graph_type(img, learner):
    """Use fastai model on each image to predict types of pages
    """
    class_names = {
        "0": ["approp"],
        "1": ["bar"],
        "2": ["bardot"],
        "3": ["box"],
        "4": ["dot"],
        "5": ["hist"],
        "6": ["other"],
        "7": ["pie"],
        "8": ["text"],
        "9": ["violin"]
    }
    
    pred_class,pred_idx,outputs = learner.predict(img)
    
    if pred_idx.sum().tolist() == 0: #if there is no predicted class 
        #(=no class over threshold) give out class with highest prediction probability
        highest_pred = str(np.argmax(outputs).tolist())
        pred_class = class_names[highest_pred]
    else: 
        pred_class = pred_class.obj #extract class name as text
        
    return(pred_class)



In [6]:
paper_id = "10.1007%2bs00393-016-0190-3"
year = '2016'
detect_graph_types_from_iiif(paper_id, year, learn)

{'bar': 0,
 'pie': 0,
 'hist': 0,
 'bardot': 0,
 'box': 0,
 'dot': 0,
 'violin': 0,
 'paper_id': '10.1007/s00393-016-0190-3',
 'year': '2016'}

### Predict number of pages with each graph type for all PDFs

In [12]:
pdf_folder = 'C:\Datenablage\charite_dashboard\PDFs'

In [13]:
paper_list = []
for root, dirs, files in os.walk(pdf_folder):
    for filename in files:
        paper_dict = {"paper_id": filename[:-4].replace("+", "%2b"),
               "year": root[-4:]}
        paper_list.append(paper_dict)
        
paper_table = pd.DataFrame(paper_list)

In [14]:
paper_table.iloc[1990:]

,paper_id,year
1990,10.1111%2bclr.12718,2015
1991,10.1111%2bclr.12723,2015
1992,10.1111%2bcodi.13031,2015
1993,10.1111%2bcpr.12208,2015
1994,10.1111%2bcrj.12344,2015
...,...,...
21714,10.7717%2bpeerj.8607,2020
21715,10.7717%2bpeerj.9384,2020
21716,10.7759%2bcureus.7305,2020
21717,alz.12121,2020


In [15]:
#as the processing takes very long, run one year at a time
year = "2020"
paper_table_filtered = paper_table[paper_table.year == year]

In [16]:
paper_table_filtered

,paper_id,year
17568,10.1001%2bjama.2020.10155,2020
17569,10.1001%2bjama.2020.10690,2020
17570,10.1001%2bjama.2020.10831,2020
17571,10.1001%2bjama.2020.15922,2020
17572,10.1001%2bjama.2020.16909,2020
...,...,...
21714,10.7717%2bpeerj.8607,2020
21715,10.7717%2bpeerj.9384,2020
21716,10.7759%2bcureus.7305,2020
21717,alz.12121,2020


In [54]:
print(year)
barzooka_results_list = [] 
for index, row in paper_table_filtered.iterrows():
    print(row['paper_id'], row['year'])
    barzooka_result = detect_graph_types_from_iiif(row['paper_id'], row['year'], learn)
    barzooka_results_list.append(barzooka_result)
    
barzooka_results = pd.DataFrame(barzooka_results_list)  
barzooka_results.to_csv("..\\results\\Barzooka_2020.csv")

2020
10.1001%2bjama.2020.10155 2020
10.1001%2bjama.2020.10690 2020
10.1001%2bjama.2020.10831 2020
10.1001%2bjama.2020.15922 2020
10.1001%2bjama.2020.16909 2020
10.1001%2bjama.2020.18618 2020
10.1001%2bjama.2020.6504 2020
10.1001%2bjama.2020.7172 2020
10.1001%2bjamacardio.2020.3551 2020
10.1001%2bjamacardio.2020.6455 2020
10.1001%2bjamadermatol.2019.4835 2020
10.1001%2bjamadermatol.2020.0723 2020
10.1001%2bjamainternmed.2020.1670 2020
10.1001%2bjamanetworkopen.2020.0772 2020
10.1001%2bjamanetworkopen.2020.12469 2020
10.1001%2bjamanetworkopen.2020.25473 2020
10.1001%2bjamanetworkopen.2020.26874 2020
10.1001%2bjamanetworkopen.2020.32335 2020
10.1001%2bjamanetworkopen.2020.3862 2020
10.1001%2bjamanetworkopen.2020.6027 2020
10.1001%2bjamaneurol.2020.2340 2020
10.1001%2bjamaneurol.2020.4152 2020
10.1001%2bjamaoncol.2020.0249 2020
10.1001%2bjamaoncol.2020.0750 2020
10.1001%2bjamaoncol.2020.1796 2020
10.1001%2bjamaoncol.2020.4574 2020
10.1001%2bjamaoncol.2020.6564 2020
10.1001%2bjamapsychiatry

10.1002%2bpros.24038 2020
10.1002%2bptr.6744 2020
10.1002%2brcs.2195 2020
10.1002%2bsctm.19-0432 2020
10.1002%2bsim.8534 2020
10.1002%2bsim.8779 2020
10.1002%2buog.22040 2020
10.1002%2buog.22125 2020
10.1002%2bwps.20699 2020
10.1002%2bwps.20714 2020
10.1002%2bwps.20718 2020
10.1002%2bwps.20765 2020
10.1007%2b112_2020_17 2020
10.1007%2b978-1-0716-0648-3_4 2020
10.1007%2b978-1-0716-0696-4_28 2020
10.1007%2b978-1-0716-0696-4_5 2020
10.1007%2b978-1-0716-0696-4_8 2020
10.1007%2b978-1-0716-0704-6_6 2020
10.1007%2b978-1-0716-0755-8_18 2020
10.1007%2b978-3-030-48457-6_9 2020
10.1007%2b978-981-15-1792-1_2 2020
10.1007%2bs00018-020-03604-w 2020
10.1007%2bs00052-020-0280-9 2020
10.1007%2bs00053-020-00430-3 2020
10.1007%2bs00053-020-00436-x 2020
10.1007%2bs00053-020-00465-6 2020
10.1007%2bs00056-019-00206-5 2020
10.1007%2bs00059-020-04897-0 2020
10.1007%2bs00059-020-04930-2 2020
10.1007%2bs00059-020-04941-z 2020
10.1007%2bs00059-020-04967-3 2020
10.1007%2bs00062-019-00874-1 2020
10.1007%2bs00062-0

10.1007%2bs00381-020-04832-y 2020
10.1007%2bs00381-020-04944-5 2020
10.1007%2bs00384-020-03721-9 2020
10.1007%2bs00391-020-01700-x 2020
10.1007%2bs00391-020-01734-1 2020
10.1007%2bs00391-020-01805-3 2020
10.1007%2bs00392-019-01593-w 2020
10.1007%2bs00392-020-01598-w 2020
10.1007%2bs00392-020-01633-w 2020
10.1007%2bs00392-020-01636-7 2020
10.1007%2bs00392-020-01650-9 2020
10.1007%2bs00392-020-01652-7 2020
10.1007%2bs00392-020-01657-2 2020
10.1007%2bs00392-020-01669-y 2020
10.1007%2bs00392-020-01672-3 2020
10.1007%2bs00392-020-01709-7 2020
10.1007%2bs00392-020-01712-y 2020
10.1007%2bs00392-020-01719-5 2020
10.1007%2bs00392-020-01732-8 2020
10.1007%2bs00392-020-01739-1 2020
10.1007%2bs00392-020-01747-1 2020
10.1007%2bs00392-020-01771-1 2020
10.1007%2bs00392-020-01773-z 2020
10.1007%2bs00393-019-00743-9 2020
10.1007%2bs00393-020-00759-6 2020
10.1007%2bs00393-020-00767-6 2020
10.1007%2bs00393-020-00786-3 2020
10.1007%2bs00393-020-00789-0 2020
10.1007%2bs00393-020-00794-3 2020
10.1007%2bs003

10.1007%2bs10554-020-02048-4 2020
10.1007%2bs10554-020-02061-7 2020
10.1007%2bs10557-020-06963-5 2020
10.1007%2bs10557-020-07040-7 2020
10.1007%2bs10561-020-09832-5 2020
10.1007%2bs10571-020-00857-8 2020
10.1007%2bs10571-020-00872-9 2020
10.1007%2bs10620-020-06328-w 2020
10.1007%2bs10637-020-00910-9 2020
10.1007%2bs10637-020-00916-3 2020
10.1007%2bs10654-020-00638-z 2020
10.1007%2bs10654-020-00679-4 2020
10.1007%2bs10719-020-09936-w 2020
10.1007%2bs10741-020-09963-7 2020
10.1007%2bs10803-020-04457-9 2020
10.1007%2bs10822-020-00310-4 2020
10.1007%2bs10875-019-00728-y 2020
10.1007%2bs10875-020-00782-x 2020
10.1007%2bs10877-020-00534-7 2020
10.1007%2bs10877-020-00578-9 2020
10.1007%2bs10899-020-09948-z 2020
10.1007%2bs10903-019-00968-5 2020
10.1007%2bs10916-020-01600-y 2020
10.1007%2bs10995-020-02937-z 2020
10.1007%2bs11013-020-09679-1 2020
10.1007%2bs11033-020-05428-0 2020
10.1007%2bs11060-019-03382-x 2020
10.1007%2bs11060-020-03640-3 2020
10.1007%2bs11060-020-03647-w 2020
10.1007%2bs110

10.1016%2bj.heliyon.2020.e05421 2020
10.1016%2bj.hlc.2020.05.095 2020
10.1016%2bj.ijid.2020.05.014 2020
10.1016%2bj.ijid.2020.08.063 2020
10.1016%2bj.isci.2020.100841 2020
10.1016%2bj.isci.2020.101539 2020
10.1016%2bj.isci.2020.101551 2020
10.1016%2bj.isci.2020.101683 2020
10.1016%2bj.jaad.2020.03.058 2020
10.1016%2bj.jaci.2020.02.001 2020
10.1016%2bj.jaci.2020.03.005 2020
10.1016%2bj.jaci.2020.03.029 2020
10.1016%2bj.jaci.2020.03.046 2020
10.1016%2bj.jaci.2020.04.059 2020
10.1016%2bj.jaci.2020.05.020 2020
10.1016%2bj.jaci.2020.05.041 2020
10.1016%2bj.jaci.2020.06.008 2020
10.1016%2bj.jaci.2020.06.009 2020
10.1016%2bj.jaci.2020.06.028 2020
10.1016%2bj.jaci.2020.07.036 2020
10.1016%2bj.jaci.2020.08.027 2020
10.1016%2bj.jaci.2020.10.015 2020
10.1016%2bj.jaci.2020.11.008 2020
10.1016%2bj.jamda.2020.06.013 2020
10.1016%2bj.jcf.2020.06.001 2020
10.1016%2bj.jclinepi.2020.08.003 2020
10.1016%2bj.jclinepi.2020.09.022 2020
10.1016%2bj.jcmgh.2020.04.002 2020
10.1016%2bj.jcmgh.2020.08.012 2020
10

...

10.1186%2bs13049-020-00756-3 2020
10.1186%2bs13049-020-00787-w 2020
10.1186%2bs13049-020-0707-2 2020
10.1186%2bs13052-020-00870-z 2020
10.1186%2bs13052-020-00939-9 2020
10.1186%2bs13054-019-2711-3 2020
10.1186%2bs13054-020-02941-3 2020
10.1186%2bs13054-020-03194-w 2020
10.1186%2bs13054-020-03275-w 2020
10.1186%2bs13054-020-03401-8 2020
10.1186%2bs13054-020-03405-4 2020
10.1186%2bs13054-020-2810-1 2020
10.1186%2bs13058-020-01283-w 2020
10.1186%2bs13058-020-01309-3 2020
10.1186%2bs13058-020-01348-w 2020
10.1186%2bs13059-020-02072-6 2020
10.1186%2bs13063-019-3928-9 2020
10.1186%2bs13063-019-4041-9 2020
10.1186%2bs13063-020-04321-2 2020
10.1186%2bs13063-020-04375-2 2020
10.1186%2bs13063-020-04384-1 2020
10.1186%2bs13063-020-04443-7 2020
10.1186%2bs13063-020-04516-7 2020
10.1186%2bs13063-020-04626-2 2020
10.1186%2bs13063-020-04654-y 2020
10.1186%2bs13063-020-04700-9 2020
10.1186%2bs13063-020-04755-8 2020
10.1186%2bs13063-020-04848-4 2020
10.1186%2bs13063-020-04973-0 2020
10.1186%2bs13072-02

10.1371%2bjournal.pone.0238021 2020
10.1371%2bjournal.pone.0238509 2020
10.1371%2bjournal.pone.0238759 2020
10.1371%2bjournal.pone.0239206 2020
10.1371%2bjournal.pone.0239210 2020
10.1371%2bjournal.pone.0239231 2020
10.1371%2bjournal.pone.0239386 2020
10.1371%2bjournal.pone.0239598 2020
10.1371%2bjournal.pone.0239853 2020
10.1371%2bjournal.pone.0240446 2020
10.1371%2bjournal.pone.0240634 2020
10.1371%2bjournal.pone.0240719 2020
10.1371%2bjournal.pone.0240892 2020
10.1371%2bjournal.pone.0241480 2020
10.1371%2bjournal.pone.0241497 2020
10.1371%2bjournal.pone.0241724 2020
10.1371%2bjournal.pone.0242062 2020
10.1371%2bjournal.pone.0242197 2020
10.1371%2bjournal.pone.0242263 2020
10.1371%2bjournal.pone.0242805 2020
10.1371%2bjournal.pone.0243147 2020
10.1371%2bjournal.pone.0243711 2020
10.1371%2bjournal.pone.0244148 2020
10.1371%2bjournal.ppat.1008263 2020
10.1371%2bjournal.ppat.1008461 2020
10.1371%2bjournal.ppat.1008870 2020
10.1371%2bjournal.ppat.1008902 2020
10.14218%2bJCTH.2020.00054 2

10.3389%2bfcimb.2020.601834 2020
10.3389%2bfcvm.2020.549392 2020
10.3389%2bfcvm.2020.599923 2020
10.3389%2bfcvm.2020.602137 2020
10.3389%2bfendo.2019.00787 2020
10.3389%2bfendo.2020.00052 2020
10.3389%2bfendo.2020.00355 2020
10.3389%2bfendo.2020.00368 2020
10.3389%2bfgene.2020.00951 2020
10.3389%2bfimmu.2019.02588 2020
10.3389%2bfimmu.2019.02911 2020
10.3389%2bfimmu.2019.03096 2020
10.3389%2bfimmu.2019.03113 2020
10.3389%2bfimmu.2020.00096 2020
10.3389%2bfimmu.2020.00179 2020
10.3389%2bfimmu.2020.00256 2020
10.3389%2bfimmu.2020.00300 2020
10.3389%2bfimmu.2020.00308 2020
10.3389%2bfimmu.2020.00413 2020
10.3389%2bfimmu.2020.00469 2020
10.3389%2bfimmu.2020.00524 2020
10.3389%2bfimmu.2020.00531 2020
10.3389%2bfimmu.2020.00578 2020
10.3389%2bfimmu.2020.00586 2020
10.3389%2bfimmu.2020.00596 2020
10.3389%2bfimmu.2020.00654 2020
10.3389%2bfimmu.2020.00716 2020
10.3389%2bfimmu.2020.00813 2020
10.3389%2bfimmu.2020.00980 2020
10.3389%2bfimmu.2020.01059 2020
10.3389%2bfimmu.2020.01091 2020
10.3389

10.3390%2bdiagnostics10060430 2020
10.3390%2bdiagnostics10080574 2020
10.3390%2bdiagnostics10090653 2020
10.3390%2bdiagnostics10090732 2020
10.3390%2bdiagnostics10110929 2020
10.3390%2bdiagnostics10110990 2020
10.3390%2bdiagnostics10121027 2020
10.3390%2bdiagnostics10121084 2020
10.3390%2bdiseases8020021 2020
10.3390%2bgenes11111275 2020
10.3390%2bijerph17010341 2020
10.3390%2bijerph17030863 2020
10.3390%2bijerph17031097 2020
10.3390%2bijerph17041401 2020
10.3390%2bijerph17072577 2020
10.3390%2bijerph17082714 2020
10.3390%2bijerph17082860 2020
10.3390%2bijerph17093200 2020
10.3390%2bijerph17103530 2020
10.3390%2bijerph17113792 2020
10.3390%2bijerph17114122 2020
10.3390%2bijerph17124229 2020
10.3390%2bijerph17124514 2020
10.3390%2bijerph17155573 2020
10.3390%2bijerph17165739 2020
10.3390%2bijerph17176349 2020
10.3390%2bijerph17196945 2020
10.3390%2bijerph17207379 2020
10.3390%2bijerph17228682 2020
10.3390%2bijerph17239030 2020
10.3390%2bijerph17249186 2020
10.3390%2bijms21010221 2020
10

FileNotFoundError: [Errno 2] No such file or directory: '..\\results\\Barzooka_2020.csv'

In [24]:
barzooka_results = pd.DataFrame(barzooka_results_list) 
barzooka_results

,bar,pie,hist,bardot,box,dot,violin,paper_id,year
0,0,0,0,0,0,0,0,10.1001/jama.2014.15770,2015
1,0,0,0,0,0,0,0,10.1001/jama.2014.5985,2015
2,0,0,0,0,0,0,0,10.1001/jama.2015.0846,2015
3,0,0,0,0,0,0,0,10.1001/jama.2015.15734,2015
4,0,0,0,0,0,0,0,10.1001/jama.2015.4668,2015
...,...,...,...,...,...,...,...,...,...
1989,0,0,1,0,0,0,0,10.1007/978-1-4939-8931-7_10,2018
1990,0,0,0,0,0,0,0,10.1007/978-3-319-65798-1_40,2018
1991,0,0,0,0,0,0,0,10.1007/978-3-319-77932-4_29,2018
1992,0,0,0,0,0,0,0,10.1007/978-3-319-91439-8_3,2018


In [56]:
barzooka_results.to_csv("../../results/Barzooka_2020.csv")

In [31]:
/Barzooka_" + year + ".csv")